In [130]:
import numpy as np
import pandas as pd

# loading the data from local file
data = pd.read_csv('dataset8.csv')
data.head()

,bpm,pace,time,temperature,gap,dist_quater,count_quater,steps_quater,puls_quater,weight_yeqr
0,149,357,4305,20,2,372590,27,1160574,60,77.1
1,147,363,4403,21,4,372480,27,1172636,60,77.1
2,149,366,4468,24,3,372320,27,1184919,60,77.1
3,153,374,5205,29,2,381430,27,1189566,60,77.2
4,151,356,6564,22,3,363030,26,1181979,60,77.2


In [131]:
print(data.shape)
print(data.isna().sum().sum())

(270, 10)
0


In [139]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   bpm           270 non-null    int64  
 1   pace          270 non-null    int64  
 2   time          270 non-null    int64  
 3   temperature   270 non-null    int64  
 4   gap           270 non-null    int64  
 5   dist_quater   270 non-null    int64  
 6   count_quater  270 non-null    int64  
 7   steps_quater  270 non-null    int64  
 8   puls_quater   270 non-null    int64  
 9   weight_yeqr   270 non-null    float64
dtypes: float64(1), int64(9)
memory usage: 21.2 KB


In [133]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('bpm', axis=1),
    data.bpm,
    test_size=0.15,
    random_state=42
)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [134]:
X_train.mean(axis = 0)

array([0.28722123, 0.28163802, 0.5748216 , 0.03582482, 0.5601904 ,
       0.60779557, 0.64834798, 0.35633188, 0.50295973])

In [135]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

lgb = LGBMRegressor(random_state=42)

# Сетка параметров
param_grid = {
    'n_estimators': [200],
    'learning_rate': [0.05],
    'max_depth': [5],
}

# GridSearchCV
model = GridSearchCV(
    estimator=lgb,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=4,         # 4-кратная кросс-валидация
    n_jobs=-1,    # параллельно на всех ядрах
    verbose=2
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

print("Best params:", model.best_params_)
print("Best RMSE:", -model.best_score_)
print('-----')
print("Test RMSE = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Train RMSE = %.4f" % np.sqrt(mean_squared_error(y_train, y_train_pred)))

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023842 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 407
[LightGBM] [Info] Number of data points in the train set: 229, number of used features: 9
[LightGBM] [Info] Start training from score 154.868996
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

C:\Users\frvit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\frvit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [138]:
best_rf = model.best_estimator_   # достаём лучший RandomForestRegressor
importances = best_rf.feature_importances_


for i, val in enumerate(importances):
    print(f"Feature {i}: {val*10:.0f}")

Feature 0: 2320
Feature 1: 1720
Feature 2: 1190
Feature 3: 1070
Feature 4: 1180
Feature 5: 1080
Feature 6: 2200
Feature 7: 560
Feature 8: 960


In [140]:
data_valid = pd.read_csv('dataset8_valid.csv')

X_valid, X_x, y_valid, y_y = train_test_split(
    data.drop('bpm', axis=1),
    data.bpm,
    test_size=0.01,
    random_state=42
)

X_valid = scaler.transform(X_valid)
y_valid_pred = model.predict(X_valid)

print("Valid RMSE = %.4f" % np.sqrt(mean_squared_error(y_valid, y_valid_pred)))

Valid RMSE = 2.3726


C:\Users\frvit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
